In [5]:
import sys
sys.path.append('/Users/Ziqi/Desktop/developer/mgwr/')
from mgwr.gwr import GWR
from mgwr.FastGWR import FastGWR
from mgwr.sel_bw import Sel_BW
import pandas as pd
import numpy as np

In [6]:
zillow = pd.read_csv("zillow_10k.csv",sep=',')
zillow.head()
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [7]:
%%time 
fast_rslt = FastGWR(coords,y,X).fit(nproc=4) #FastGWR fitting with optimal bw seaching

mpiexec -n 4 python /Users/Ziqi/Desktop/developer/mgwr/mgwr/FastGWR_mpi.py -data temp_data.csv -out fastGWRResults.csv
CPU times: user 294 ms, sys: 32.2 ms, total: 326 ms
Wall time: 1min 38s


In [8]:
%%time 
bw = Sel_BW(coords,y,X).search(bw_min=45) #MGWR fitting with optimal bw seaching
rslt = GWR(coords,y,X,bw=122).fit()

CPU times: user 2min 24s, sys: 49.3 s, total: 3min 13s
Wall time: 3min 20s


In [9]:
%%time
fast_rslt = FastGWR(coords,y,X,bw=122).fit(nproc=4) #FastGWR fitting with defined bandwidth

mpiexec -n 4 python /Users/Ziqi/Desktop/developer/mgwr/mgwr/FastGWR_mpi.py -data temp_data.csv -out fastGWRResults.csv -bw 122
CPU times: user 283 ms, sys: 186 ms, total: 470 ms
Wall time: 7.94 s


In [10]:
%%time
rslt = GWR(coords,y,X,bw=122).fit() #MGWR fitting with defined bandwidth

CPU times: user 35.5 s, sys: 5.73 s, total: 41.3 s
Wall time: 41.8 s


In [11]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)

1041.6607662675278
1041.6607662675278


In [12]:
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)

124374.05181029711
124374.0518102971


In [13]:
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)

13051.99722906043
13051.99722906043


In [14]:
np.allclose(fast_rslt.params, rslt.params)

True

In [15]:
np.allclose(fast_rslt.bse, rslt.bse)

True

In [16]:
np.allclose(fast_rslt.predy, rslt.predy.reshape(-1))

True